In [6]:
import pandas as pd

train_file = '/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/train.csv'
test_file = '/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/test.csv'
output_file = '/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/predictions.csv'

train = pd.read_csv( train_file )
test = pd.read_csv( test_file )

# remove NaNs from Open
test.loc[ test.Open.isnull(), 'Open' ] = 1
# remove rows with 0 sales from train
train = train.loc[train.Sales > 0]

print("group by store, day of week and promo and calculate mean sales for each group")
means = train.groupby([ 'Store', 'DayOfWeek', 'Promo' ])['Sales'].mean()
print(means.head(6))

print("reset index to get a dataframe with 4 columns")
means = means.reset_index()
print(means.head(6))

print("merge with test dataframe to get sales predictions")
test = pd.merge(test, means, on = ['Store','DayOfWeek','Promo'], how='left')
test.fillna(train.Sales.mean(), inplace=True)
print(test.head())

test[[ 'Id', 'Sales' ]].to_csv( output_file, index = False )


group by store, day of week and promo and calculate mean sales for each group
Store  DayOfWeek  Promo
1      1          0        3893.306452
                  1        5852.708333
       2          0        3835.285714
                  1        5364.597222
       3          0        3746.746032
                  1        5073.736111
Name: Sales, dtype: float64
reset index to get a dataframe with 4 columns
   Store  DayOfWeek  Promo        Sales
0      1          1      0  3893.306452
1      1          1      1  5852.708333
2      1          2      0  3835.285714
3      1          2      1  5364.597222
4      1          3      0  3746.746032
5      1          3      1  5073.736111
merge with test dataframe to get sales predictions
   Id  Store  DayOfWeek        Date  Open  Promo StateHoliday  SchoolHoliday  \
0   1      1          4  2015-09-17     1      1            0              0   
1   2      3          4  2015-09-17     1      1            0              0   
2   3      7       

In [5]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [8]:
len(train), len(test)

(1017209, 41088)

In [2]:
import pandas as pd
import numpy as np

In [3]:
first = pd.read_csv("/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/rf1.csv")
second = pd.read_csv("/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/early_stop50_3001.csv")
third = pd.read_csv("/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/early_stop100_5000.csv")


In [4]:
f = np.asarray(first["Sales"])
s = np.asarray(second["Sales"])
t = np.asarray(third["Sales"])

In [5]:
def model_ensemble(*predictions, **prediction_mode):
    '''
    Perform Model Averaging or ensembling

    (predictions instances for the different models, string) -> (array)
    Possible options prediction_modes : ['binary', 'probs_binary, probs_multi_class, 'continuous']

    binary : 0/1 binary predictions
    probs : likelihood / probabilities
    continuous : float / integers - continuous

    *prediction takes the prediction results from either a given set of instances of a prediction or
    all the preictions for all the intances in the prediction set
    **prediction_mode takes the prediction mode

    binary
    ------
    bin_one
    >>> array([0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0])
    bin_two
    >>> array([0 0 0 1 0 0 1 1 1 1 0 0 0 0 1 0 1 1 0 0 1 0 1 1 1 1 0 0 0 0])
    bin_three
    >>> array([0 0 0 1 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 1 0 1 0 0 0 1 1 1 1 0])
    bin_four
    >>> array([1 0 0 0 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1 0 0 0 1 1 0 1 0 1 0 1])
    model_ensemble(bin_one, bin_two, bin_three, bin_four, mode = 'binary')
    >>> array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0])

    probs_binary / probs_multi_class
    --------------------------------
    probs_one
    >>>[[ 0.14130402  0.63774128]
     [ 0.50817373  0.98390654]
     [ 0.11188119  0.7557338 ]
     [ 0.22295128  0.21470534]
     [ 0.0471746   0.18134759]]
    probs_two
    >>> [[ 0.43791867  0.2953648 ]
     [ 0.17505953  0.73938007]
     [ 0.55570754  0.91328122]
     [ 0.57098714  0.9477655 ]
     [ 0.03664345  0.21077864]]
    probs_three
    >>> [[ 0.94495286  0.31086103]
     [ 0.73768356  0.97090608]
     [ 0.47792986  0.06297078]
     [ 0.12999474  0.3582617 ]
     [ 0.22364166  0.59997857]]
    model_ensemble(probs_one, probs_two, probs_three, mode = 'probs')
    >>> array([[ 0.94285177,  0.45720821],
       [ 0.57108943,  0.3851022 ],
       [ 0.52443183,  0.89947945],
       [ 0.56761841,  0.13814018],
       [ 0.58913088,  0.45394568]])


    continuous
    ----------
    cont_one
    >>> array([ 2, 42, 93, 52,  5, 50, 31, 54,  5, 74])
    cont_teo
    >>> array([61, 57, 19, 15, 30,  8, 34,  2, 65, 69])
    cont_three
    >>> array([35, 32, 18, 15, 56, 52, 51, 44,  6, 12])
    model_ensemble(cont_one, cont_two, cont_three, mode = 'continuous')
    >>> array([ 33.33,  35.33,  29.33,  33.33, 43.67,  36. ,  57.33,  48.33, 21. ,  46.67])
    '''

    for name, value in prediction_mode.items():
        if value is 'probs':
            # model averaging - Soft Ensembling
            model_avg = np.array(predictions)
            return model_avg.mean(axis = 0)

        majority_vote = []
        if value is 'binary':
            # perform model averaging or majority vote - Hard Ensembling
            # advisable to have odd number of models to break ties
            for i in xrange(len(predictions)):
                majority_vote.append(np.argmax(np.bincount((np.vstack((predictions)))[:,i])))
            return np.array(majority_vote)

        if value is 'continuous':
            # perform model averaging
            model_avg = (np.vstack((predictions))).mean(axis=0)
            return model_avg

In [6]:
#print model_ensemble(f, s, t, mode = "continuous")
ens = model_ensemble(f, s, t, mode = "continuous")
np.savetxt("/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/foo.csv", model_ensemble(f, s, t, mode = "continuous"), delimiter=",")

In [7]:
import csv
i = np.asarray(first["Id"])
rows = zip(i,ens)

with open('/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/test.csv', 'wb') as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)


In [8]:
print ens

[ 4054.79592512  4534.83617937  3521.9830144  ...,  7115.76145669
  6400.22837647  7107.4609143 ]


In [9]:
(f[0] + s[0] + t[0])/3

TypeError: 'file' object has no attribute '__getitem__'

In [10]:
(f[1] + s[1] + t[1])/3

TypeError: 'file' object has no attribute '__getitem__'

In [11]:
b1 = pd.read_csv("/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/Ensemble/Final_Average/rf1.csv")
b2 = pd.read_csv("/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/Ensemble/Final_Average/test.csv")
b3 = pd.read_csv("/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/Ensemble/Final_Average/TreeBooster.csv")

In [12]:
(b1["Sales"][0] + b2["Sales"][0])/2

4064.8718502421407

In [13]:
ens2 = model_ensemble(b1["Sales"], b2["Sales"], b3["Sales"] ,mode = "continuous")


In [14]:
ens2

array([ 4088.06790016,  4530.61823917,  3552.7640192 , ...,  7047.09860892,
        6262.65450196,  7073.92121907])

In [15]:
import csv
i = np.asarray(first["Id"])
rows = zip(i,ens2)

with open('/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/Ensemble/Final_Average/final.csv', 'wb') as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)


In [16]:
ens3 = model_ensemble(b1["Sales"], b2["Sales"] ,mode = "continuous")
ens4 = model_ensemble(b2["Sales"], b3["Sales"] ,mode = "continuous")
ens5 = model_ensemble(b1["Sales"], b3["Sales"] ,mode = "continuous")

In [20]:
import csv
i = np.asarray(first["Id"])
rows = zip(i,ens3)

with open('/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/Ensemble/Final_Average/fina2.csv', 'wb') as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)


In [21]:
import csv
i = np.asarray(first["Id"])
rows = zip(i,ens4)

with open('/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/Ensemble/Final_Average/fina3.csv', 'wb') as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)


In [22]:
import csv
i = np.asarray(first["Id"])
rows = zip(i,ens5)

with open('/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/Ensemble/Final_Average/fina4.csv', 'wb') as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)


In [23]:
b1 = pd.read_csv("/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/Ensemble/Final_Average/fina2.csv")
b2 = pd.read_csv("/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/Ensemble/Final_Average/fina3.csv")
b3 = pd.read_csv("/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/Ensemble/Final_Average/fina4.csv")

ens6 = model_ensemble(b1["Sales"], b2["Sales"], b3["Sales"] ,mode = "continuous")

In [24]:
import csv
i = np.asarray(first["Id"])
rows = zip(i,ens6)

with open('/Users/jalemrajrohit/Desktop/Data Science/Kaggle/Rossman Stores/Ensemble/Final_Average/fina5.csv', 'wb') as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)